# 股票预测模型工作流

---
### 工作流说明
1.  **阶段零 (Setup)**: 导入库、加载配置。
2.  **阶段一 (Data Pipeline)**: 独立运行。负责处理并保存数据，生成 L2 特征数据缓存。
3.  **阶段二 (Model Pipeline)**: 独立运行。包含三个子步骤：
    - **2.1 HPO**: 自动调参。
    - **2.2 (预处理)**: 智能地加载或生成 L3 预处理数据缓存
    - **2.3 (模型训练)**: 使用 L3 缓存进行高效的模型训练。
    - **2.4 (评估)**: 对训练结果进行聚合与可视化。

## 0. 通用设置与导入

In [1]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# 设置 Matplotlib 样式 (可以在 main_train 外部设置)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# --- 动态导入 main_train ---
project_root = str(Path().resolve())
if project_root not in sys.path:
    sys.path.append(project_root)

# 从我们统一的核心引擎中导入所有需要的函数和模块
from main_train import (
    run_load_config_and_modules,
    run_all_data_pipeline,
    run_preprocess_l3_cache,
    run_hpo_train,
    run_all_models_train,
    run_performance_evaluation,
    run_results_visualization
)

# --- 加载全局配置和模块 ---
# 在 Notebook 的生命周期中，我们只加载一次
config, modules = run_load_config_and_modules()

if not (config and modules):
    raise RuntimeError("环境初始化失败，请检查配置文件路径和模块导入。")

d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\main_train.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


--- 正在初始化环境：加载配置与模块... ---
INFO: 底层并行计算库线程数已设置为: 4
INFO: 项目模块导入成功。
SUCCESS: 配置已从 'configs/config.yaml' 加载。


# **阶段一：数据准备与特征工程**

In [2]:
if config and modules:
    # 调用核心引擎中的数据处理函数
    # 它会自动处理 API 初始化、动态日期计算和数据保存
    run_all_data_pipeline(config, modules)

=== 阶段一：数据准备与特征工程 ===
INFO: 正在尝试登录 Baostock...
login success!
INFO: Baostock API 登录成功。
INFO: 未在配置中提供有效的 Tushare Token，将跳过 Tushare 相关数据。
开始执行数据管道协调任务...
INFO: 正在根据 'end_date' 和 'data_lookback_years' 动态计算 'start_date'...
      计算得出的 start_date 为: 2010-09-30，已更新到本次运行的全局配置中。
INFO: 特征文件已存在于 data\processed\000100.SZ\4cac08e82ad6\features.pkl，跳过 TCL科技 (000100.SZ) 的数据处理。
INFO: 特征文件已存在于 data\processed\000426.SZ\5617fe5e7af3\features.pkl，跳过 兴业矿业 (000426.SZ) 的数据处理。
INFO: 特征文件已存在于 data\processed\002083.SZ\5d6a6b153d0e\features.pkl，跳过 孚日股份 (002083.SZ) 的数据处理。
INFO: 特征文件已存在于 data\processed\000150.SZ\5634b34f3878\features.pkl，跳过 宜华健康 (000150.SZ) 的数据处理。
INFO: 特征文件已存在于 data\processed\300013.SZ\4cac08e82ad6\features.pkl，跳过 新宁物流 (300013.SZ) 的数据处理。
INFO: 特征文件已存在于 data\processed\300242.SZ\30376337ba8e\features.pkl，跳过 佳云科技 (300242.SZ) 的数据处理。
INFO: 特征文件已存在于 data\processed\002006.SZ\26378c150652\features.pkl，跳过 精功科技 (002006.SZ) 的数据处理。
INFO: 特征文件已存在于 data\processed\300242.SZ\30376337ba8e\features.pkl，跳过 佳云科技 (3

# **阶段二：模型训练与评估**

### 2.1 数据预加载与全局预处理 (L3 缓存)

In [ ]:
if config and modules:
    # 调用核心引擎中的预处理函数
    # force_reprocess=True/False 可以方便地控制是否重建缓存
    global_data_cache = run_preprocess_l3_cache(config, modules, force_reprocess=False)

=== 阶段 2.1：数据预加载与全局预处理 (L3 缓存) ===
INFO: 发现已存在的 L3 预处理缓存。正在从 data\processed\_preprocessed_cache.joblib 加载...


### 2.2 超参数优化

In [ ]:
# 同样，可以通过一个简单的开关来控制是否运行
RUN_HPO = False

if RUN_HPO and config and modules and 'global_data_cache' in locals():
    # 调用核心引擎中的 HPO 函数
    # HPO 的结果会自动更新到内存中的 config 字典里
    run_hpo_train(config, modules, global_data_cache)

### 2.3 模型训练

In [ ]:
if config and modules and 'global_data_cache' in locals():
    # 调用核心引擎中的模型训练函数
    # 同样可以通过开关控制是否强制重训
    all_ic_history = run_all_models_train(
        config, 
        modules, 
        global_data_cache, 
        force_retrain_base=False, 
        force_retrain_fuser=False
    )

### 2.4 结果聚合、评估与可视化

In [ ]:
if config and modules and 'all_ic_history' in locals() and all_ic_history:
    # 调用核心引擎中的评估和可视化函数
    evaluation_summary, backtest_summary, final_eval_df = run_performance_evaluation(config, modules, all_ic_history)
    
    # 只有在有结果时才进行可视化
    if evaluation_summary is not None or backtest_summary is not None:
        run_results_visualization(config, modules, evaluation_summary, backtest_summary, final_eval_df)